In [6]:
pip install langchain openai wikipedia numexpr

Defaulting to user installation because normal site-packages is not writeable
  Using cached wikipedia-1.4.0.tar.gz (27 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11704 sha256=5d52b5a31beb226bea668e7715ef610303b8c5d542dc00e8b0ef385e11ad72f5
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\79\1d\c8\b64e19423cc5a2a339450ea5d145e7c8eb3d4aa2b150cde33b
Successfully built wikipedia
Note: you may need to restart the kernel to use updated packages.


  DEPRECATION: Building 'wikipedia' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'wikipedia'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [ ]:
# ==============================
# 운영체제 기능 사용 모듈
# (환경변수 읽기 위해 필요)
# ==============================
import os

# ==============================
# .env 파일에 저장된 환경변수를
# 자동으로 불러오는 라이브러리
# ==============================
from dotenv import load_dotenv


# ==============================
# .env 파일 로드
# → OPENAI_API_KEY 같은 비밀키를
#   파이썬 프로그램에서 사용할 수 있게 함
# ==============================
load_dotenv()
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")

In [16]:
# ==============================
# LangChain에서 OpenAI Chat 모델
# 사용을 위한 클래스 불러오기
# ==============================
from langchain_classic.chat_models import ChatOpenAI


# ==============================
# GPT 모델 객체 생성
# ==============================
            # temperature = 0
                # → 출력 결과 랜덤성 제거
                # → 항상 최대한 정확하고 일관된 답변 생성
llm=ChatOpenAI(temperature=0,
               # 사용할 OpenAI 모델 지정
                # gpt-4.1-mini = 빠르고 저렴한 GPT-4 계열 모델
               model_name='gpt-4.1-mini')



# ==============================
# Agent 관련 기능 불러오기
# ==============================

# 도구(tool) 로드 함수
from langchain_classic.agents import load_tools
# 에이전트 생성 함수
from langchain_classic.agents import initialize_agent
# 에이전트 유형(Enum)
from langchain_classic.agents import AgentType



# ==============================
# 사용할 도구 설정
# ==============================

    # 사용할 도구 목록
     # 위키피디아 검색 도구 (정보 조회용)
     # 수학 계산 전용 도구
      # 도구 내부에서 사용할 LLM
tools=load_tools(['wikipedia','llm-math'],llm=llm)


# ==============================
# Agent 생성
# ==============================
  # 사용할 도구 목록
  # 사용할 GPT 모델
agent=initialize_agent(tools,
                       llm,
                           # 에이전트 동작 방식 지정
                            # ZERO_SHOT_REACT_DESCRIPTION
                            #
                            # 의미:
                            # - ZERO_SHOT : 예시 없이도 스스로 판단
                            # - REACT     : Reasoning + Action
                            #               (생각 → 도구사용 → 결과반영)
                            # - DESCRIPTION : 도구 설명 기반 판단
                            #
                            # 즉:
                            # 질문을 보고
                            # "검색이 필요하면 wikipedia"
                            # "계산이 필요하면 llm-math"
                            # 자동 선택
                       agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                        # 에이전트 설명 (도구 선택 힌트)
                       description="계산이 필요할때 사용",
                        # verbose=True
                        # → Agent의 사고과정과
                        # → 어떤 도구를 사용했는지
                        # → 터미널에 모두 출력
                       verbose=True)

# ==============================
# Agent 실행 (질문 입력)
# ==============================
 # 질문 내용:
    # 1. 정진운 데뷔 연도 검색 필요 (wikipedia)
    # 2. 2026 - 데뷔연도 계산 필요 (llm-math)
    #
    # → Agent가 자동으로 도구 선택 및 실행
agent.run("배우 박시후의 대표작은?")



> Entering new AgentExecutor chain...
Thought: 박시후의 대표작을 알아보기 위해 박시후에 대해 검색해보자.
Action: wikipedia
Action Input: 박시후

Observation: Page: Park Si-hoo
Summary: Park Si-hoo (Korean: 박시후; born February 15, 1977), birth name Park Pyeong-ho, is a South Korean actor. He began his entertainment career as an underwear model and stage actor, then made his official television debut in 2005. After several years in supporting roles, Park rose to fame in 2010 with the popular romantic comedy series Queen of Reversals and Prosecutor Princess. This was followed by leading roles in period drama The Princess' Man (2011) and thriller film Confession of Murder (2012).

Page: Babel (TV series)
Summary: Babel (Korean: 바벨) is a 2019 South Korean television series starring Park Si-hoo and Jang Hee-jin. The series aired on TV Chosun from January 27 to March 24, 2019 on Saturdays and Sundays at 22:50 (KST).

Page: Lee Da-in (actress, born 1992)
Summary: Lee Ra-yoon (Korean: 이라윤; born Im Yoo-kyung on November 5

"배우 박시후의 대표작으로는 드라마 'Queen of Reversals', 'Prosecutor Princess', 'The Princess' Man'과 영화 'Confession of Murder' 등이 있습니다."